<a href="https://colab.research.google.com/github/SiyiGit/github_actions/blob/main/cnn_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install opendatasets
!pip install opendatasets

In [ ]:
#import libraries
import opendatasets as od
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
import cv2
import random
import os
from os import listdir
from PIL import Image
import seaborn as sns

In [ ]:
#kaggle dataset download
od.download("https://www.kaggle.com/datasets/vipoooool/new-plant-diseases-dataset")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: alicjasosialuk
Your Kaggle Key: ··········


100%|██████████| 2.70G/2.70G [00:25<00:00, 112MB/s]


In [ ]:
#just for info{"username":"alicjasosialuk","key":"fa69bfa4724b0a10006ba687db303759"}

In [ ]:
#path to datasets
data_dir_train = './new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train'
data_dir_val = './new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid'
data_dir_test = './new-plant-diseases-dataset/test/test'

In [ ]:
#listing the folders
os.listdir(data_dir_test)

['AppleScab3.JPG',
 'CornCommonRust1.JPG',
 'AppleScab1.JPG',
 'PotatoEarlyBlight1.JPG',
 'TomatoEarlyBlight5.JPG',
 'PotatoHealthy2.JPG',
 'TomatoYellowCurlVirus1.JPG',
 'PotatoEarlyBlight5.JPG',
 'TomatoEarlyBlight1.JPG',
 'PotatoHealthy1.JPG',
 'TomatoHealthy2.JPG',
 'CornCommonRust2.JPG',
 'TomatoEarlyBlight6.JPG',
 'TomatoEarlyBlight4.JPG',
 'PotatoEarlyBlight2.JPG',
 'TomatoHealthy1.JPG',
 'AppleCedarRust3.JPG',
 'TomatoYellowCurlVirus5.JPG',
 'TomatoHealthy3.JPG',
 'AppleCedarRust4.JPG',
 'TomatoYellowCurlVirus6.JPG',
 'TomatoHealthy4.JPG',
 'PotatoEarlyBlight3.JPG',
 'TomatoYellowCurlVirus2.JPG',
 'AppleScab2.JPG',
 'AppleCedarRust2.JPG',
 'TomatoEarlyBlight2.JPG',
 'TomatoYellowCurlVirus4.JPG',
 'AppleCedarRust1.JPG',
 'PotatoEarlyBlight4.JPG',
 'CornCommonRust3.JPG',
 'TomatoEarlyBlight3.JPG',
 'TomatoYellowCurlVirus3.JPG']

CNN model

In [ ]:
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, MaxPooling2D, Conv2D, Activation, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from sklearn import metrics
import tensorflow as tf


In [ ]:
## Defining batch specfications
batch_size = 100
img_height = 100
img_width = 100

In [ ]:
## loading training set
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                  zoom_range=0.2,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  fill_mode='nearest')
train_data = train_datagen.flow_from_directory(data_dir_train,
                                              batch_size=batch_size,
                                              target_size=(img_height, img_width),
                                              class_mode = 'categorical',
                                              shuffle=True)

Found 70295 images belonging to 38 classes.


In [ ]:
## loading validation set
valdata_generator=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
val_data = valdata_generator.flow_from_directory(data_dir_val,
                                            batch_size=batch_size,
                                            target_size=(img_height, img_width),
                                            class_mode = 'categorical',
                                            shuffle=False)

Found 17572 images belonging to 38 classes.


In [ ]:
## loading test set
valdata_generator=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_data = valdata_generator.flow_from_directory(data_dir_test,
                                            batch_size=batch_size,
                                            target_size=(img_height, img_width),
                                            class_mode = 'categorical',
                                            shuffle=False)

Found 0 images belonging to 0 classes.


In [ ]:
## Defining model layers
inputs = Input(shape = (28, 28,3), name = "Input")

first_layer = Conv2D(filters = 32,
                     kernel_size = (5, 5),
                     padding = 'valid',
                     activation = 'relu')

second_layer = MaxPooling2D(pool_size = (2, 2))

third_layer = Dropout(rate = 0.2)

fourth_layer = Flatten()

fifth_layer = Dense(units = 128,
                    activation = 'relu')

output_layer = Dense(units = 10,
                     activation='softmax')

x=first_layer(inputs)
x=second_layer(x)
x=third_layer(x)
x=fourth_layer(x)
x=fifth_layer(x)

outputs=output_layer(x)

cnn_model = Model(inputs = inputs, outputs = outputs)


In [ ]:
cnn_model.compile(loss=['categorical_crossentropy','categorical_crossentropy', 'categorical_crossentropy'], # loss function
                  optimizer='adam',                # gradien descent algorithm
                  metrics=['accuracy'],           # accuracy metric
                 loss_weights=[1, 0.3, 0.3])


In [ ]:
print(cnn_model.summary())

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 28, 28, 3)]       0         
                                                                 
 conv2d_4 (Conv2D)           (None, 24, 24, 32)        2432      
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 12, 12, 32)        0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 12, 12, 32)        0         
                                                                 
 flatten_2 (Flatten)         (None, 4608)              0         
                                                                 
 dense_3 (Dense)             (None, 128)               589952    
                                                           

In [ ]:
train_number=train_data.samples
valid_number=val_data.samples

In [ ]:
#training_history = cnn_model.fit(train_data,
#                                 validation_data= val_data,
#                                 epochs = 50)

train_history = cnn_model.fit(train_data,
                              validation_data=val_data,
                              steps_per_epoch=train_number // batch_size,
                              epochs=50,
                              validation_steps=valid_number // batch_size,
                              verbose=2)
#                              callbacks=[reduce_lr])

Epoch 1/50


InvalidArgumentError: Graph execution error:

Detected at node model_1/dense_3/Relu defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.

In [ ]:
train_acc = training_history.history['accuracy']
val_acc = training_history.history['val_accuracy']

train_loss = training_history.history['loss']
val_loss = training_history.history['val_loss']

learning_rate = training_history.history['lr']

In [ ]:
plt.xlabel('Epochs')
plt.ylabel('Accuracy')


plt.plot(np.arange(1 , 11, 1),
         train_acc,
         label = 'Training Accuracy',
         color = 'blue')


plt.plot(np.arange(1 , 11, 1),
         val_acc,
         label = 'Validation Accuracy',
         color = 'red')

plt.legend()

plt.show()


In [ ]:
test_pred = cnn_model.predict(X_test)

test_pred_class = test_pred.argmax(axis = 1)
y_test_class = y_test.argmax(axis = 1)

print(metrics.classification_report(y_test_class, test_pred_class))

